In [ ]:
import os
import pickle
import pandas as pd
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer


os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
llm = LLM(model="Qwen/Qwen2.5-7B-Instruct/", tensor_parallel_size=4, max_model_len=8192, gpu_memory_utilization=0.85)
df = pd.read_json("ScaleQM+_test.json")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct/")
texts = []
for i in range(len(df)):
    system1 = """You are a mathematics teacher reviewing a solution that may contain logical gaps or jumps in reasoning.
- Your task is to:
1. Carefully examine the logical flow of the solution and identify any points where the reasoning jumps ahead without sufficient explanation or intermediate steps. For each such logical gap, specify exactly between which two consecutive steps it occurs.
2. Provide the missing reasoning or intermediate step(s) that would bridge the gap, including any necessary explanations or equations.
Note that the solution may contain multiple logical gaps, or it might be logically complete. The steps in the solution are labeled from Step 0 (problem statement) to Step N.

- Please format your response as follows:
For each missing step, output:
Missing Step X:
The missing step should be placed between Step Y and Step Y+1.
The missing step is:
[Write the missing step here directly]

If there are no missing steps, please output:
No missing steps.

Just provide the fill locations and content according to the format, without giving extra explanations or the complete solution after filling. Don't include confirmations like "No other missing steps" after filling in.

- Example Output:
Missing Step 1:  
The missing step should be placed between Step 5 and Step 6.  
The missing step is:  
We isolate the variable x by subtracting 5 from both sides of the equation:  
7x + 5 = 26  
=> 7x = 21

Missing Step 2:  
The missing step should be placed between Step 8 and Step 9.  
The missing step is:  
We divide both sides of the equation by 7 to solve for x:  
7x = 21  
=> x = 3
"""
    prompt1 = f"""Here is the solution.

{df.iloc[i]["messages"][1]["content"][22:-24]}"""
    messages = [
        {"role": "system",
         "content": system1},
        {"role": "user",
         "content": prompt1},
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    texts.append(text)

outputs = llm.generate(
    texts,
    SamplingParams(
    temperature=0,
    max_tokens=1024,
    skip_special_tokens=True
)
)

results = []
for i in range(len(outputs)):
    results.append(outputs[i].outputs[0].text)

with open('results-7b.pkl', 'wb') as f:
    pickle.dump(results, f)